<a href="https://colab.research.google.com/github/yebyyy/English-Premier-League-2023-2024-Prediction/blob/main/Scraping.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### This code is for scraping the data for the current season of English Premier League

In [220]:
import requests

In [221]:
import time

In [222]:
import pandas as pd

In [223]:
standings_url = "https://fbref.com/en/comps/9/Premier-League-Stats"

In [240]:
data = requests.get(standings_url)

In [225]:
from bs4 import BeautifulSoup

In [226]:
soup = BeautifulSoup(data.text)

#### Now we need give the soup object something to select.
Since everything is inside the table html object, we need to select it and then get everything anchor tags for the clubs.

In [241]:
data

<Response [429]>

In [232]:
# selecting the table using css selector
standings_table = soup.select('tables.stats_table')[0] # .stats_table is the class name

IndexError: list index out of range

In [ ]:
# find all the <a>(the clubs) in the standings table
links = standings_table.find_all("a") # find_all only finds tags

In [ ]:
# links should actually contain all the links
links = [l.get("href") for l in links]

In [ ]:
# links should actually contain the links that have clubs
links = [l for l in links if "/squads/" in l]
links

In [ ]:
# since the links only have the subdomain, we should include the first half of the url as well
team_urls = [f"https://fbref.com{l}" for l in links]
team_urls

#### Using Pandas and Requests to extract match stats: Using a single team as an example

In [ ]:
team_url_first = team_urls[0]
team_url_first

In [ ]:
first_team_data = requests.get(team_url_first)

Now we use pandas for the parsing and read only the table called scores and fixtures since we only care about the data there

In [ ]:
matches_first_team = pd.read_html(first_team_data.text, match="Scores & Fixtures")

Since this is a list, we want a pandas dataFrame, therefore we get the first element from this list

In [ ]:
matches_first_team = matches_first_team[0]

#### Now we also care about the shooting data, so we continue using the same team for the example

Since the shooting page is now just a URL in an anchor in the webpage of the club page we were working on, we find all the links and keep the one that has shooting in it

In [ ]:
soup_for_first_team = BeautifulSoup(first_team_data.text)

In [ ]:
link_for_first_team = soup_for_first_team.find_all("a")

In [ ]:
link_for_first_team = [l.get("href") for l in link_for_first_team]

In [ ]:
link_for_first_team = [l for l in link_for_first_team if l and "all_comps/shooting/" in l]
link_for_first_team

In [ ]:
first_team_shooting_data = requests.get(f"https://fbref.com{link_for_first_team[0]}")

In [ ]:
first_team_shooting_frame = pd.read_html(first_team_shooting_data.text, match='Shooting')
first_team_shooting_frame = first_team_shooting_frame[0]

#### Cleaning and merging data in pandas

We don't want the multilevel index, so just drop that

In [ ]:
first_team_shooting_frame.columns = first_team_shooting_frame.columns.droplevel()

In [ ]:
first_team_shooting_frame.head()

In [ ]:
first_team_frame = matches_first_team.merge(first_team_shooting_frame[["Date", "Sh", "SoT", "Dist", "FK", "PK", "PKatt"]], on="Date")
first_team_frame.head()

This just added additional columns in the end of the frame

In [ ]:
matches_first_team.shape

In [ ]:
first_team_shooting_frame.shape

### Now we should scrape the data for multiple seasons and for all of the teams existed in the English Premier League

In [ ]:
years = list(range(2024, 2021, -1))
years

In [ ]:
# several DataFrames and each DataFrame contains the match log for a team
all_matches = []

In [ ]:
for year in years:
  # get the links to the teams through the table.stats_table
  data = requests.get(standings_url)
  soup = BeautifulSoup(data.text)
  table = soup.select("table.stats_table")[0]
  links = standings_table.find_all("a")
  links = [l.get("href") for l in links]
  links = [l for l in links if "/squads/" in l]

  previous_season = soup.select("a.prev")[0].get("href")
  standings_url = f"https://fbref.com{previous_season}"

  team_urls = [f"https://fbref.com{l}" for l in links]
  for team in team_urls:
    team_name = team.split("/")[-1].replace("-Stats", "").replace("-", " ")
    data = requests.get(team)
    # read the table that has the Scores and Fixtures
    matches = pd.read_html(data.text, match="Scores & Fixtures")[0]
    # get the link to the scoring data
    soup = BeautifulSoup(data.text)
    links = [l.get("href") for l in soup.find_all('a')]
    links = [l for l in links if l and "all_comps/shooting/" in l]
    data = requests.get(f"https://fbref.com{links[0]}")
    shooting = pd.read_html(data.text, match='Shooting')[0]
    shooting.columns = shooting.columns.droplevel()

    try:
      team_data = matches.merge(shooting[["Date", "Sh", "SoT", "Dist", "FK", "PK", "PKatt"]], on="Date")
    # for some teams the shooting table is not available
    except ValueError:
      continue

    # filter by competition and add column indicating years and team names
    team_data = team_data[team_data["Comp"] == "Premier League"]
    team_data["Season"] = year
    team_data["Team"] = team_name
    all_matches.append(team_data)
    time.sleep(3)




In [ ]:
match_df = pd.concat(all_matches)
match_df.columns = [c.lower() for c in match_df.columns]

In [ ]:
match_df.to_csv("matches.csv")